In [44]:
import logging
import sys
import nest_asyncio

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

nest_asyncio.apply()

In [45]:
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
    Response,
    
)
import pinecone
import openai
import streamlit as st
from llama_index.vector_stores import PineconeVectorStore
from llama_index.llms import OpenAI
import pandas as pd
from llama_index.evaluation import FaithfulnessEvaluator

pd.set_option("display.max_colwidth", 0)

In [46]:
openai.api_key = st.secrets.openai_key
api_key = "2bcecfc8-35f9-44dc-9459-ab0cc2781b29"
pinecone.init(api_key=api_key, environment="gcp-starter")
pinecone_index = pinecone.Index("tailwind-hugging")

In [47]:
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
service_context = ServiceContext.from_defaults(llm=OpenAI(model="gpt-3.5-turbo", temperature=0.5, system_prompt="You are an expert on TailwindCSS and your job is to answer technical questions. Assume that all questions are related to TailwindCSS. Keep your answers technical and based on facts – do not hallucinate features."))
index = VectorStoreIndex.from_vector_store(vector_store=vector_store, service_context=service_context)

In [48]:
evaluator_faithfullness = FaithfulnessEvaluator(service_context=service_context)

In [56]:
chat_engine = index.as_chat_engine(chat_mode="react", verbose=True)
query_str = "What class should I use from TailwindCSS that lets me create grids with n equally sized columns?"
# query_engine = index.as_query_engine()
response_vector = chat_engine.query(query_str)
eval_result = evaluator_faithfullness.evaluate_response(response=response_vector)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Thought: The user is asking about creating grids with equally sized columns using TailwindCSS. I can use the query_engine_tool to find the appropriate class for this.
Action: query_engine_tool
Action Input: {'input': 'TailwindCSS class for equally sized columns in grids'}
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK

In [57]:
def display_eval_faith_df(response: Response, eval_result: str) -> None:
    if response.source_nodes == []:
        print("no response!")
        return
    eval_df = pd.DataFrame(
        {
            "Query": query_str,
            "Response": str(response),
            "Evaluation Result": "Pass" if eval_result.passing else "Fail",
        },
        index=[0],
    )
    eval_df = eval_df.style.set_properties(
        **{
            "inline-size": "600px",
            "overflow-wrap": "break-word",
        },
        subset=["Query", "Response"]
    )
    display(eval_df)

In [58]:
display_eval_faith_df(response_vector, eval_result)

,Query,Response,Evaluation Result
0,What class should I use from TailwindCSS that lets me create grids with n equally sized columns?,"To create grids with n equally sized columns in TailwindCSS, you can use the class `grid-cols-{n}`, where `{n}` represents the number of equally sized columns you want to create.",Pass
